In [2]:
import numpy as np
from scipy.optimize import fsolve

In [3]:
import matplotlib
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from scipy import signal
from scipy.interpolate import interp1d
from matplotlib.ticker import AutoMinorLocator
from scipy.signal import filter_design as fd

matplotlib.rc('text', usetex='True')
#matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rcParams.update({'font.size': 30})#,'family':'monospace'})

                                   
#matplotlib.rc('font',**{'family':'sans-serif','sans-serif':['fira']})  
matplotlib.rcParams['xtick.direction'] = 'in'
matplotlib.rcParams['ytick.direction'] = 'in'
matplotlib.rcParams['xtick.top'] = True
matplotlib.rcParams['ytick.right'] = True

matplotlib.rcParams['pgf.texsystem'] = 'pdflatex'
matplotlib.rcParams.update({'pgf.rcfonts' : False})




matplotlib.rcParams['axes.linewidth'] = 4
matplotlib.rcParams['xtick.major.size'] = 12
matplotlib.rcParams['xtick.major.width'] = 4
matplotlib.rcParams['xtick.minor.size'] = 10
matplotlib.rcParams['xtick.minor.width'] = 3

matplotlib.rcParams['ytick.major.size'] = 12
matplotlib.rcParams['ytick.major.width'] = 4
matplotlib.rcParams['ytick.minor.size'] = 10
matplotlib.rcParams['ytick.minor.width'] = 3


In [49]:
def spectrogram_full(t,h):
    N = 2*512
    nf = 2*N
    dt = 0.05/(nf)
    freq = np.fft.fftfreq(nf, d=dt)

    ffpeuq = interp1d(t, h)
    tshift = min(t)
    fnyq = max(freq)
    bl, al = signal.butter(2, 1200.0/fnyq,'low')
    bh, ah = signal.butter(2, 25.0/fnyq,'high')
    window = signal.kaiser(nf,2.5)
    window = signal.blackman(nf)
    t2 = np.linspace(tshift,0.05+tshift,nf)
    tarr = []
    fttm = []
    j = 0
    while(max(t2) < max(t)):
        amp = ffpeuq(t2)
        amp1 = signal.filtfilt(bh, ah, amp)
        amp2 = signal.filtfilt(bl, al, amp1)
        amp3 = amp2*window
        fftamp  = (np.abs(fft(amp3))[0:N]**2) /np.sum(window)**2
        fttm.append([])
        fttm[j] = fftamp
        tarr.append((max(t2)+min(t2))*0.5)
        tshift = tshift + 0.001
        t2 = np.linspace(tshift,0.05+tshift,nf)
        j = j+1
    rt = fttm,freq[0:N],tarr
    return rt

In [39]:
def plot_amp(t,hc,hp, title=None,savename=None):
    f1,ax =  plt.subplots(figsize=(25,10),ncols=2,nrows=1, sharex="col",sharey="all")
    ax[0].plot(t,hc,label=r'$h_\times$',lw=2)
    ax[1].plot(t,hp,label=r'$h_+$',lw=2)
    f1.subplots_adjust(hspace=0.,wspace=0.0)
    
    ax[0].legend(frameon=False,handlelength=0.5)
    ax[1].legend(frameon=False,handlelength=0.5)
    
    ax[0].set_ylabel(r'$\mathrm{Amplitude} \ \mathrm{[cm]}$')
    ax[1].set_xlabel(r'$\mathrm{Time} \ \mathrm{[ms]}$')
    ax[0].set_xlabel(r'$\mathrm{Time} \ \mathrm{[ms]}$')
    if(title != None):
        f1.suptitle(title)
    if(savename != None):
        plt.savefig(savename,bbox_inches='tight')
    
    

In [77]:
def plot_spectrogram(fil):
    t,axx,ayy,azz,axy,axz,ayz = np.loadtxt(fil, unpack = True)
    ap_po = axx-ayy
    ac_po = 2*axy
    
    ap_eq = azz-ayy
    ac_eq = -2*ayz
    print(min(t),max(t),max(t)-min(t))
    t = t - min(t)
    jet = cm = plt.get_cmap('jet')
    v = np.linspace(-6,0,50)
    
    f1,ax=plt.subplots(figsize=(20,10),ncols=2,nrows=1,sharex="row",sharey="row")

    ffc,f,t1=spectrogram_full(t,ap_po)
    ffp,f,t1=spectrogram_full(t,ac_po)
    ft=np.log10((np.transpose(ffp)**2+np.transpose(ffc)**2+1e-13)/0.05)
    cb=ax[0].contourf(np.array(t1)*1000,f,ft,v,cmap=jet,extend="both")
    f1.subplots_adjust(hspace=0.,wspace=0.0)

    ffc,f,t1 =spectrogram_full(t,ap_eq)
    ffp,f,t1 =spectrogram_full(t,ac_eq)
    ft=np.log10((np.transpose(ffp)**2+np.transpose(ffc)**2+1e-13)/0.05)
    cb=ax[1].contourf(np.array(t1)*1000,f,ft,v,cmap=jet,extend="both")
    
    
    ax[0].set_ylim([25,1250])
    ax[0].set_xlim([min(t1)*1000,max(t1)*1000])
    #ax[0].set_xticks([100,200,300,400,500])
    ax[0].set_yticks([100,250,400,550,700,850,1000,1150])
    ax[0].xaxis.set_minor_locator(AutoMinorLocator(4))
    ax[0].yaxis.set_minor_locator(AutoMinorLocator(3))
    
    ax[1].set_ylim([25,1250])
    ax[1].set_xlim([min(t1)*1000,max(t1)*1000])
    #ax[1].set_xticks([100,200,300,400,500])
    ax[1].set_yticks([100,250,400,550,700,850,1000,1150])
    ax[1].xaxis.set_minor_locator(AutoMinorLocator(4))
    ax[1].yaxis.set_minor_locator(AutoMinorLocator(3))
    
    
    title = fil[:-4]
    title = title.replace("_","-")
    svname = title + "-spec.png"
    f1.colorbar(cb,ax=ax[1],ticks=[0,-1,-2,-3,-4,-5,-6],pad=0.007,aspect=30)
    plt.savefig(svname,bbox_inches='tight')

In [67]:
def plot_pol(fil):
    t,axx,ayy,azz,axy,axz,ayz = np.loadtxt(fil, unpack = True)
    ap = axx-ayy
    ac = 2*axy
    title = fil[:-4]
    title = title.replace("_","-")
    svname = title + ".png"
    plot_amp(t*1000,ac,ap,svname)

In [68]:
def plot_eq(fil):
    t,axx,ayy,azz,axy,axz,ayz = np.loadtxt(fil, unpack = True)
    ap = azz-ayy
    ac = -2*ayz
    title = fil[:-4]
    title = title.replace("_","-")
    svname = title + ".png"
    plot_amp(t*1000,ac,ap,title,svname)